In [1]:
import h5py
import glob
import numpy as np
import random
from scipy.spatial.distance import squareform
from settings import *

In [2]:
%matplotlib widget
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d, Axes3D
# pick which random coords to display:
f = h5py.File('/data/Schema/intact/fsaverage6_adj.h5', 'r')
dispcoordsLH = f['lhinflatedcoords'][:]
randcoords = random.sample(range(len(dispcoordsLH[1])), 5000)
dispcoordsLH = dispcoordsLH[:,randcoords]
dispcoordsRH = f['rhinflatedcoords'][:]
dispcoordsRH = dispcoordsRH[:,randcoords]

fig1 = plt.figure()

ISCfile = 'ISC.h5'
f = h5py.File(ISCpath+ISCfile, 'r')

for idx,task in enumerate(['DM','TP']):
    if 'cat' not in ISCfile:
        ISC = np.nan_to_num(np.mean(f[task]['ISC_persubj'][:],axis=1))
    else:
        ISC = np.nan_to_num(np.mean(f['ISC_persubj'][:],axis=1))
        task = 'all'
    # For SUMA visualization:
    np.savetxt(ISCpath+task+'ISC.lh.dset',ISC[:len(ISC)//2])
    np.savetxt(ISCpath+task+'ISC.rh.dset',ISC[len(ISC)//2:])
    dispISCLH = ISC[:len(ISC)//2][randcoords] # For LH
    dispISCRH = ISC[len(ISC)//2:][randcoords] # For RH
    # plot ISC values across cortex
    axrh = fig1.add_subplot(2,2,idx+1, projection='3d')
    x,y,z = dispcoordsRH
    brainrh = axrh.scatter3D(x,y,z,c=dispISCRH, cmap='bwr')
    fig1.colorbar(brainrh)
    axrh.set_title(task + ' RH')
    
    axlh = fig1.add_subplot(2,2,idx+3, projection='3d')
    x,y,z = dispcoordsLH
    brainlh = axlh.scatter3D(x,y,z,c=dispISCLH, cmap='bwr');
    fig1.colorbar(brainlh)
    axlh.set_title(task + ' LH')

FigureCanvasNbAgg()

In [3]:
fig2 = plt.figure()

Ages = []
Sexes = []
for s in f['subord'][:]:
    fs = h5py.File(prepath + s.split('/')[-1],'r')
    Ages.append(fs['Pheno']['Age'].value)
    Sexes.append(fs['Pheno']['Sex'].value)
Agesr = np.round(Ages)
Agesu = list(set(Agesr))

for idx,task in enumerate(['DM','TP']):
    if 'cat' not in ISCfile:
        ISC_persubj = np.mean(np.nan_to_num(f[task]['ISC_persubj'][:])>0.1,axis=0)
    else:
        ISC_persubj = np.mean(np.nan_to_num(f['ISC_persubj'][:])>0.1,axis=0)
        task = 'all'
    
    # ISC_persubj = np.nanmean(f[task]['ISC_persubj'][:],axis=0)
    ISCam = []
    ISCas = []
    for a in Agesu:
        idxs = list(np.where(Agesr==a))[0]
        ISCam.append(np.mean(ISC_persubj[idxs]))
        ISCas.append(np.std(ISC_persubj[idxs]))
    ISCsm = []
    ISCss = []
    for s in np.int64([0,1]):
        idxs = list(np.where(Sexes==s))[0]
        ISCsm.append(np.mean(ISC_persubj[idxs]))
        ISCss.append(np.std(ISC_persubj[idxs])) 
    # plot per-subject ISC values by age
    ax = fig2.add_subplot(2,2,idx+1)
    ax.errorbar(Agesu, ISCam, ISCas, marker='o', linestyle='None')
    plt.ylabel('ISC')
    plt.xlabel('Age')
    ax.set_title(task + ' ISC by age')
    # plot per-subject ISC values by sex
    ax = fig2.add_subplot(2,2,idx+3)
    ax.errorbar([0,1], ISCsm, ISCss, linestyle='None', marker='^')
    ax.xaxis.set_ticks([0, 1])
    ax.xaxis.set_ticklabels(('Male','Female'))
    ax.set_title(task + ' ISC by sex')
    plt.ylabel('ISC')
fig2.tight_layout()
fig2.savefig(figurepath+'ISC.png')

FigureCanvasNbAgg()

In [4]:
def sortby(items):
    import itertools
    pairstmp = list(itertools.combinations(range(len(items)), 2))
    sort_index = np.argsort(items)
    pairs = np.full(np.shape(pairstmp),np.nan)
    for sidx,i in enumerate(sort_index):
        for idx,p in enumerate(pairstmp):
            if i in p:
                pairs[idx][p.index(i)] = sidx
    # Sort pairs: [(0,1),(0,2),...(3,4)]
    sort = []
    for idx in range(len(sort_index)):
        pidx = [idxp for idxp,element in enumerate(pairs) if idx in element]
        # check if idx already in sort:
        pidx = list(set(pidx).difference(set(pidx).intersection(sort)))
        sort.extend(pidx[i] for i in np.argsort(np.sum(pairs[pidx],axis=1)))
    return sort

In [5]:
# plot matrix of per-subject ISCs
import itertools
import seaborn as sns; #sns.set()
#fig3 = plt.figure()
sort_age = sortby(Agesr)
sort_sex = sortby(Sexes)

ROIs = ['RSC','A1']
figs={}
for roi in ROIs:
    figs[roi] = plt.figure()
    cbar_ax = figs[roi].add_axes([.91, .3, .03, .4])
    for idx,task in enumerate(['DM','TP']):
        #iscs_ages = squareform(np.mean(np.nan_to_num(f[task]['isc_pairs'][:])>0.1,axis=1)[sort_age])
        #iscs_sexes = squareform(np.mean(np.nan_to_num(f[task]['isc_pairs'][:])>0.1,axis=1)[sort_sex])
        iscs_ages = squareform(f[task]['isc_pairs'][roi][:][sort_age])
        iscs_sexes = squareform(f[task]['isc_pairs'][roi][:][sort_sex])
        ax = figs[roi].add_subplot(2,2,idx+1)
        sns.heatmap(data=iscs_ages, ax=ax,xticklabels=np.sort(Agesr),yticklabels=np.sort(Agesr),#annot=True,
               vmin=0,vmax=0.4,cbar=(idx==0),cbar_ax=None if idx else cbar_ax)
        ax.set_title(task + ' '+roi+' ISC by age')
        ax = figs[roi].add_subplot(2,2,idx+3)
        sns.heatmap(data=iscs_sexes, ax=ax,xticklabels=np.sort(Sexes),yticklabels=np.sort(Sexes),#annot=True,
               vmin=0,vmax=0.4,cbar=(idx==0),cbar_ax=None if idx else cbar_ax)
        ax.set_title(task + ' '+roi+' ISC by sex')

    figs[roi].tight_layout(rect=[0, 0, .9, 1])
    figs[roi].savefig(figurepath+'ISC_'+roi+'.png')

FigureCanvasNbAgg()

/home/scohen/.conda/envs/scohen_brainiak_env_0.8/lib/python3.6/site-packages/matplotlib/figure.py:2267: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  warnings.warn("This figure includes Axes that are not compatible "


FigureCanvasNbAgg()

/home/scohen/.conda/envs/scohen_brainiak_env_0.8/lib/python3.6/site-packages/matplotlib/figure.py:2267: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  warnings.warn("This figure includes Axes that are not compatible "


In [41]:
subord = glob.glob(prepath+'sub*.h5')
#subord = subord[0:5] # for testing!
ROIs = ['RSC','A1']
figs2 = {}
for task in ['DM','TP']:
    figs2[task] = plt.figure(figsize=(8,4))
    ROIdata = {}
    for roi in ROIs:
        ROIdata[roi] = []
    for subidx, sub in enumerate(subord):
        f = h5py.File(sub, 'r')
        for roi in ROIs:
            if roi in list(f[task].keys()):
                ROIdata[roi].append(f[task][roi][:])
    for ridx, roi in enumerate(ROIs):
        ROIdata[roi] = np.stack(ROIdata[roi])
        t = ROIdata[roi].shape[2]
        corr = []
        for sub in range(len(ROIdata[roi])):
            tmp = np.corrcoef(ROIdata[roi][sub,:,:],
                    np.mean(ROIdata[roi][[x for x in range(len(ROIdata[roi])) if x!=sub],:,:],axis=0),
                               rowvar=False)
            corr.append(np.mean(np.stack([tmp[:t,t:],tmp[t:,:t]]),axis=0))
        ax = figs2[task].add_subplot(1,2,ridx+1)
        sns.heatmap(data=np.mean(np.stack(corr),axis=0),vmin=-0.1,vmax=0.1)#,cbar=(idx==0),cbar_ax=None if ridx else cbar_ax)
        ax.set_title(task + ' '+roi)
    figs2[task].savefig(figurepath+'xcorr_'+task+'.png')

FigureCanvasNbAgg()

FigureCanvasNbAgg()

In [ ]:
%matplotlib widget
fig = plt.figure()
#ax = plt.axes(projection='3d')
ax = fig.gca(projection='3d')

x,y,z = dispcoordsRH
brain = ax.scatter3D(x,y,z,c=dispISCRH, cmap='bwr');
plt.colorbar(brain)